In [1]:
!pip install transformers
!pip install datasets
!pip install torch

In [2]:
!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

^C


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset

dataset = load_dataset('xsum', save_infos=True)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

Using custom data configuration default
Reusing dataset xsum (C:\Users\Chang\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# 데이터셋 본문, 요약 예시 출력
dummy_data = dataset['train'][:5]
document = dummy_data['document']
summary = dummy_data['summary']
print(document)
print(summary)

['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - bu

In [5]:
!pip install nltk
!pip install pandas

In [5]:
import re


def preprocessing_data(data):
    x_data = data['document']
    x_data = re.sub('\n', '', x_data)
    y_data = data['summary']
    
    x_data = tokenizer(x_data, truncation=True, max_length=1024, return_tensors='pt')
    y_data = tokenizer(y_data, truncation=True, max_length=512, return_tensors='pt')
    
    return x_data, y_data



In [6]:
input_data = []
decode_data = []

for data in dataset['train']:
    x_data, y_data = preprocessing_data(data)
    input_data.append(x_data)
    decode_data.append(y_data)
    
print(input_data[0])
print(decode_data[0])

{'input_ids': tensor([[    0,   133,   455,   701,     9,  1880,    11, 10793,  6192,     6,
            65,     9,     5,   911,  2373,  2132,     6,    16,   202,   145,
         11852,     4, 22026,  2456,   173,    16,  2256,    11, 10034,  1758,
             8,   171,  3197,    11,   221,  1942,   428,  1672,  6867,  1091,
          7340,  2132,    30,  2934,   514,     4, 12667,  5069,    15,     5,
          3072,  3673, 42656,   652, 10044,   528,     7,  1880,    23,     5,
           226,  9708,  1054, 16376,   625, 21491,     4, 10787,  1252,     8,
          6028,   268,    58,  2132,    30,  5681,    11, 10793,  6192,    71,
             5,  1995, 30084, 41031,  9725,    88,     5,  1139,     4, 10993,
           692, 14371, 21801,  3790,     5,   443,     7, 18973,     5,  1880,
             4,   133,  5794, 18646,    10, 17784,  2204,     6,  5681,   171,
          1861,  3611,    15,  4769,   852,   111,     5,  1049,  3482, 10675,
         17825,     4, 35689,  3398, 1

In [11]:
input_ids = input_data[0]
decode_ids = decode_data[0]

model.train()
model.forward(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'], decoder_input_ids=decode_ids['input_ids'], decoder_attention_mask=decode_ids['attention_mask'])

Seq2SeqLMOutput(loss=None, logits=tensor([[[ 35.4949,  -1.9287,  10.6009,  ...,  -5.0643,  -4.2781,   6.2286],
         [-21.8630,  -3.5075,   1.8857,  ...,  -3.8155,  -3.6385,   1.8780],
         [-10.2855,  -3.5346,   3.5774,  ...,  -3.1631,  -3.5544,   0.2572],
         ...,
         [-16.1415,  -3.6182,   3.1881,  ...,  -4.3922,  -3.6869,   1.3946],
         [ -6.9615,  -2.9209,   4.7538,  ...,  -4.0637,  -3.8369,   1.4756],
         [ -1.8730,  -2.9717,   5.5935,  ...,  -3.3243,  -4.2017,   1.9229]]],
       grad_fn=<AddBackward0>), past_key_values=((tensor([[[[ 1.1011e+00,  2.6627e+00,  4.4931e+00,  ..., -1.3612e+00,
            3.2593e+00, -2.8859e+00],
          [-1.3547e+00, -2.5358e+00, -1.8238e+00,  ...,  3.3178e+00,
           -6.1547e-01,  1.8401e+00],
          [-2.9925e+00,  3.7449e-02, -1.5862e+00,  ..., -2.2614e+00,
           -3.0554e+00, -3.2230e+00],
          ...,
          [-1.3069e+00,  6.9481e-01,  3.1492e-01,  ..., -5.3137e+00,
           -5.9088e+00,  2.9205e+

In [12]:
ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], truncation=True, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=5)
print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['My friends']
